In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('C:/Users/jcrig/Documents/FORMATION_SORBONNE/Machine_Learning_2_Faik/grand_ml_project/input_data/rating.csv')

In [5]:
df.head(5)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [6]:
df = df.drop(['timestamp'], axis=1)

In [7]:
df.head(5)

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [6]:
df.shape

(20000263, 3)

In [7]:
df.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

In [8]:
df['userId'] = df['userId'].astype('int32')
df['movieId'] = df['movieId'].astype('int32')
df['rating'] = df['rating'].astype('float32')


In [9]:
df.dtypes

userId       int32
movieId      int32
rating     float32
dtype: object

In [10]:
nan_counts = df.isna().sum()
nan_counts

userId     0
movieId    0
rating     0
dtype: int64

In [11]:
df['movieId'].nunique()

26744

In [12]:
df['userId'].nunique()

138493

In [9]:
user_splits = np.array_split(df['userId'].unique(), 10)

df_pivot_list = []
for split in user_splits:
    df_subset = df[df['userId'].isin(split)]
    df_pivot_subset = pd.pivot_table(df_subset, values='rating', index='userId', columns='movieId')
    df_pivot_list.append(df_pivot_subset)

df_pivot = pd.concat(df_pivot_list)


In [14]:
df_pivot.shape

(138493, 26744)

In [15]:
df_pivot.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,130586,130604,130614,130622,130656,130828,131110,131172,131237,131262
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_encoded = pd.read_csv('output_data\df_encoded.csv')
df_encoded.set_index('movieId', inplace = True)
df_encoded.head(5)

In [13]:
df_encoded_num = df_encoded.select_dtypes(include=[float, int])

In [15]:
def matching_genres(movie_title):
    # Trouver l'index du film cible
    i = int(df_encoded.index[df_encoded['primaryTitle'] == movie_title][0])
    print('1')
    # Sélectionner les genres du film cible
    target_genres = df_encoded_num.loc[i]
    print('2')
    # Calculer la corrélation des genres avec le film cible
    correlations = df_encoded_num.apply(lambda row: row.corr(target_genres), axis=1)
    print('3')
    # Créer un DataFrame des résultats pour les genres
    df_genre = pd.DataFrame(correlations, columns=['PearsonG'])
    print('4')
    # Ajouter les titres des films
    df_genre = df_genre.join(df_encoded[['primaryTitle']])
    print('5')
    # Trier les résultats par la corrélation des genres en ordre décroissant
    df_genre = df_genre.sort_values('PearsonG', ascending=False)
    df_genre.dropna(inplace=True)
    print('6')
    # Calculer la corrélation des notes avec le film cible
    target = df_pivot.loc[i]
    similar_to_target = df_pivot.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns=['PearsonR'])
    corr_target.dropna(inplace=True)
    corr_target = corr_target.sort_values('PearsonR', ascending=False)
    corr_target.index = corr_target.index.map(int)
    print('7')
    # Fusionner les DataFrames de corrélations de genres et de notes
    df_combined = corr_target.join(df_genre.set_index(df_genre.index), how='inner')
    print('8')
    # Trier par corrélation de PearsonR et PearsonG
    df_combined = df_combined.sort_values(['PearsonR', 'PearsonG'], ascending=False)
    print('9')
    # Afficher les 20 meilleures recommandations
    print(df_combined.head(20).to_string(index=False))

# Appeler la fonction pour un exemple
matching_genres('Dazed and Confused')


1
2
3
4
5
6


c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


7
8
9
 PearsonR  PearsonG                                  primaryTitle
      1.0  1.000000                               Made in America
      1.0  1.000000              Some Folks Call It a Sling Blade
      1.0  1.000000                                      Soapdish
      1.0  0.999999                                 Nobody's Fool
      1.0  0.999998                          The War of the Roses
      1.0  0.999996                                200 Cigarettes
      1.0  0.999996                                          Boys
      1.0  0.999993                                        Kazaam
      1.0  0.999992                                  Withnail & I
      1.0  0.999991                                       Roxanne
      1.0  0.999987                            Great Expectations
      1.0  0.999986                                        Kundun
      1.0  0.999985                         Peggy Sue Got Married
      1.0  0.999982                                Without Limits
    